# Amazon SageMaker Model Monitor

*이 노트북은 [Amazon SageMaker Model Monitor (영문 원본)](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker_model_monitor/introduction/SageMaker-ModelMonitoring.ipynb) 의 한국어 번역입니다.*

본 노트북은 아래의 방법들을 수행하는 법을 보여줍니다.
* Amazon SageMaker에서 머신 러닝 모델을 호스팅하고 추론 요청(inference requests), 결과 및 메타데이터(metadata)를 캡처
* 학습 데이터 셋을 분석하여 baseline 제약 조건(constraints) 생성
* 제약 조건에 대한 위반이 있는지 실시간 엔드포인트 모니터링

---
## Background

Amazon SageMaker는 모든 개발자 및 데이터 과학자에게 머신 러닝 모델을 신속하게 구축, 학습 및 배포할 수 있는 기능을 제공합니다. Amazon SageMaker는 전체 머신 러닝 워크 플로우를 포함하는 완전 관리형 서비스(fully-managed service)입니다. 여러분은 데이터를 준비하고 알고리즘을 선택하고 모델을 학습한 다음 배포를 위해 미세 조정 및 최적화를 수행할 할 있습니다. Amazon SageMaker를 사용하여 모델을 프로덕션에 배포하여 이전에 가능했던 것보다 적은 비용으로 예측하고 비용을 절감할 수 있습니다.

또한 re:Invent 2019에서 소개된 Amazon SageMaker Model Monitor를 사용하면 배포한 모델의 호출에 대한 입력, 출력 및 메타데이터를 캡처할 수 있으며, 데이터를 분석하고 품질을 모니터링할 수 있습니다. 즉, 모델 배포 이후에 데이터 품질이 변하는 것을 감지하기 때문에 실제 서비스 배포에 유용하게 쓰일 수 있습니다. 이 노트북 예제를 통해 여러분은 Amazon SageMaker가 이러한 기능들을 어떻게 지원하는지 배울 수 있습니다.

---
## Setup

시작하려면 다음 선수 조건들을 완료했는지 확인하세요.

* 모델을 호스팅 할 AWS 리전을 지정하세요.
* Amazon SageMaker에 Amazon Simple Storage Service (Amazon S3)의 데이터에 대한 액세스 권한을 부여하는 데 사용되는 IAM 역할 ARN이 있습니다. 필요한 권한을 미세 조정(fine tune)하는 방법은 문서를 참조하세요.
* 모델 학습에 사용된 데이터, 추가 모델 데이터 및 모델 호출에서 캡처된 데이터를 저장하는 데 사용되는 S3 버킷(bucket)을 생성하세요. 본 노트북에서는 데모 목적으로 동일한 버킷을 사용하고 있지만, 실제로는 다른 보안 정책으로 분리할 수 있습니다.

In [1]:
%%time

# Handful of configuration

import os
import boto3
import re
import json
from sagemaker import get_execution_role, session

region= boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket =  session.Session(boto3.Session()).default_bucket()
print("Demo Bucket: {}".format(bucket))
prefix = 'sagemaker/DEMO-ModelMonitor'

data_capture_prefix = '{}/datacapture'.format(prefix)
s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)
reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)
code_prefix = '{}/code'.format(prefix)
s3_code_preprocessor_uri = 's3://{}/{}/{}'.format(bucket,code_prefix, 'preprocessor.py')
s3_code_postprocessor_uri = 's3://{}/{}/{}'.format(bucket,code_prefix, 'postprocessor.py')

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))
print("Preproc Code path: {}".format(s3_code_preprocessor_uri))
print("Postproc Code path: {}".format(s3_code_postprocessor_uri))

RoleArn: arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20191206T081784
Demo Bucket: sagemaker-us-east-1-143656149352
Capture path: s3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/datacapture
Report path: s3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/reports
Preproc Code path: s3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/code/preprocessor.py
Postproc Code path: s3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/code/postprocessor.py
CPU times: user 784 ms, sys: 70.5 ms, total: 854 ms
Wall time: 1.29 s


여러분은 이 노트북의 실행 역할에 필요한 권한이 있는지 빠르게 확인할 수 있습니다. 위에서 지정한 S3 버킷에 간단한 테스트 객체(test object)를 넣습니다. 이 명령이 실패하면 버킷에 대한 `s3:PutObject` 권한을 갖도록 역할을 업데이트하고 다시 시도해 보세요.

In [2]:
# Upload some test files
boto3.Session().resource('s3').Bucket(bucket).Object("test_upload/test.txt").upload_file('test_data/upload-test-file.txt')
print("Success! You are all set to proceed.")

Success! You are all set to proceed.


# PART A: Amazon SageMaker 엔드포인트에서 실시간 추론 데이터 캡처 (Capturing real-time inference data from Amazon SageMaker endpoints)
실제로 데이터 캡처 기능을 보여주는 엔드포인트(endpoint)를 작성합니다.

### Upload the pre-trained model to Amazon S3

이 코드는 배포 준비가 된 사전 학습된 XGBoost 모델을 업로드합니다. 이 모델은 SageMaker의 XGB Churn Prediction Notebook(Churn Prediction: 고객 이탈 예측)을 사용하여 학습되었습니다. 물론 이 단계에서 여러분의 사전 학습된 모델을 사용할 수도 있습니다. Amazon S3에 사전 학습된 모델이 이미 있는 경우, s3_key를 지정하여 모델을 추가할 수 있습니다.

In [3]:
model_file = open("model/xgb-churn-prediction-model.tar.gz", 'rb')
s3_key = os.path.join(prefix, 'xgb-churn-prediction-model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

### Deploy the model to Amazon SageMaker

사전 학습된 고객 이탈 예측 모델을 배포하는 단계부터 시작하세요. 먼저, 이미지 및 모델 데이터를 사용하여 모델 객체(model)를 생성합니다.

In [4]:
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.amazon.amazon_estimator import get_image_uri

model_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://{}.s3-{}.amazonaws.com/{}/xgb-churn-prediction-model.tar.gz'.format(bucket, region, prefix)
image_uri = get_image_uri(boto3.Session().region_name, 'xgboost', '0.90-1')

model = Model(image=image_uri, model_data=model_url, role=role)

모델 데이터 품질을 모니터링하기 위해 데이터 캡처를 사용하려면 `DataCaptureConfig`라는 신규 캡처 옵션을 지정하세요. 이 구성으로 요청 페이로드(payload), 응답 페이로드 또는 둘 다 캡처할 수 있습니다. 캡처 구성은 모든 변형에 적용됩니다. 이제 배포를 진행합시다. (참고로 아래 셀은 약 10분 정도 소요됩니다.)

In [5]:
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = 'DEMO-xgb-churn-pred-model-monitor-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path)

predictor = model.deploy(initial_instance_count=1,
                instance_type='ml.m4.xlarge',
                endpoint_name=endpoint_name,
                data_capture_config=data_capture_config)

EndpointName=DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33
----------------------------------------------------------------------------------------------------------------!

## Invoke the deployed model

이제 이 엔드포인트로 데이터를 전송하여 실시간으로 추론할 수 있습니다. 이전 단계에서 데이터 캡처를 활성화했으므로, 요청 및 응답 페이로드는 일부 추가 메타 데이터와 함께 `DataCaptureConfig`에서 지정한 Amazon Simple Storage Service (Amazon S3) 위치에 저장됩니다.

이 단계는 약 2분 동안 샘플 데이터가 포함된 엔드포인트를 호출합니다. 지정된 샘플링 백분율을 기준으로 데이터가 캡처되고 데이터 캡처 옵션이 해제될 때까지 캡처가 계속됩니다.

In [6]:
from sagemaker.predictor import RealTimePredictor
import time

predictor = RealTimePredictor(endpoint=endpoint_name,content_type='text/csv')

# get a subset of test data for a quick test
!head -120 test_data/test-dataset-input-cols.csv > test_data/test_sample.csv
print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

with open('test_data/test_sample.csv', 'r') as f:
    for row in f:
        payload = row.rstrip('\n')
        response = predictor.predict(data=payload)
        time.sleep(0.5)
        
print("Done!")        

Sending test traffic to the endpoint DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33. 
Please wait...
Done!


## View captured data

이제 Amazon S3에 저장된 데이터 캡처 파일들을 나열해 보겠습니다. 호출이 발생한 시간을 기준으로 구성된 다른 기간의 다른 파일을 볼 수 있어야 합니다. Amazon S3 경로의 형식은 다음과 같습니다.

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [7]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
sagemaker/DEMO-ModelMonitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33/AllTraffic/2019/12/09/00/22-59-729-2ea5a0f1-2304-465a-a3ff-d557f69abd9e.jsonl
 sagemaker/DEMO-ModelMonitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33/AllTraffic/2019/12/09/00/24-00-242-1b766114-12af-496a-bfac-8bb7e4a87552.jsonl


다음으로 단일 캡처 파일의 내용을 확인합니다. 여기에는 Amazon SageMaker 특정 JSON 라인 형식 파일에서 캡처된 모든 데이터가 표시됩니다. 캡처한 파일의 첫 몇 줄을 살짝 살펴보세요.

In [8]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"92,0,176.3,85,93.4,125,207.2,107,9.6,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.039806101471185684","encoding":"CSV"}},"eventMetadata":{"eventId":"1ced17af-34ab-499a-ba8f-eb714295d660","inferenceTime":"2019-12-09T00:24:00Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"138,0,46.5,104,186.0,114,167.5,95,9.6,4,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.9562002420425415","encoding":"CSV"}},"eventMetadata":{"eventId":"828e06dd-80d1-4322-bb94-2594cf231076","inferenceTime":"

한 줄로 보는 대신 JSON 파일을 파싱하여 좀 더 쉽게 파악해 보실 수 있습니다.

In [9]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "92,0,176.3,85,93.4,125,207.2,107,9.6,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.039806101471185684",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "1ced17af-34ab-499a-ba8f-eb714295d660",
    "inferenceTime": "2019-12-09T00:24:00Z"
  },
  "eventVersion": "0"
}


여러분이 확인하였겠지만, 각 추론 요청은 jsonl 파일에서 한 줄로 캡처됩니다. 이 줄에는 입력과 출력이 함께 병합되어 있습니다. 이 예에서는 ContentType을 `text/csv`로 제공했으며 `observedContentType` 값에 반영됩니다. 또한 입력 및 출력 페이로드를 인코딩하는 데 사용한 인코딩을 인코딩 값으로 캡처 형식으로 노출합니다.

요약하자면, 여러분은 새로운 매개 변수를 사용하여 입력 또는 출력 페이로드를 엔드 포인트에 캡처하는 방법을 관찰하였고 Amazon S3에서 캡처된 형식이 어떻게 보이는지 관찰했습니다. 다음 파트에서 Amazon SageMaker가 Amazon S3에서 수집된 데이터를 모니터링하는 데 어떻게 도움이 되는지 계속 확인해 보겠습니다.

# PART B: Model Monitor - Baseling and continuous monitoring

Amazon SageMaker는 데이터 수집 외에도 엔드포인트에서 관찰한 데이터를 모니터링하고 평가할 수 있는 기능을 제공합니다. 
모델이 학습 당시에는 정상적으로 학습되었다 하더라도, 모델 배포 이후 여러 가지 요인으로 인해 추론 데이터셋 내 데이터의 변동이 있을 경우 모델을 그대로 두면 정확도, 정밀도 등의 지표가 줄어들 수 있습니다. 이를 모니터링하는 방법을 알아 보겠습니다.

이를 위해:

1. 실시간 트래픽을 비교할 기준을 만듭니다.
1. 기준이 준비되면 지속적으로 baseline과 평가하고 비교할 일정(schedule)을 셋업합니다.

## 1. Constraint suggestion with baseline/training dataset

모델을 학습한 학습 데이터셋은 일반적으로 좋은 데이터셋입니다. 학습 데이터셋 데이터 스키마와 추론 데이터셋 스키마는 정확히 일치해야 합니다. (예: feature의 개수와 순서가 일치)

학습 데이터셋에서 Amazon SageMaker에 일련의 baseline 제약 조건(`constraints`)을 제안하고 설명 통계(`statistics`)를 생성하여 데이터를 탐색하도록 요청할 수 있습니다. 이 노트북 예시에서는 이 예시에 포함된 사전 학습된 모델을 학습시키는 데 사용된 학습 데이터셋을 업로드합니다. 물론 여러분이 이미 학습 데이터셋을 Amazon S3에 가지고 있는 경우 경로를 직접 가리킬 수 있습니다.

- `constraints.json`: 학습 데이터셋의 유추 스키마 정보를 알려줍니다. 예를 들어 특정 feature의 결측값이 존재하지 않는 경우 completeness는 1.0입니다.
- `statistics.json`: 각 feature의 통계치(평균, 표준편차, 사분위수 등)와 고유한 값의 개수를 알려줍니다.

In [11]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))

Baseline data uri: s3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/baselining/results


In [12]:
training_data_file = open("test_data/training-dataset-with-header.csv", 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'training-dataset-with-header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

### Create a baselining job with training dataset

Amazon S3에서 학습 데이터를 준비했으므로 제약 조건을 제안(`suggest`)하는 작업(job)을 시작하세요. `DefaultModelMonitor.suggest_baseline(..)`은 Amazon SageMaker가 제공하는 모델 모니터 컨테이너(Model Monitor container)를 사용하여 제약 조건을 생성하는 `ProcessingJob`을 시작합니다.

여기에서 생성한 baseline을 기준으로 추론 데이터가 학습 데이터와 다른지 판단하게 됩니다. 

In [13]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri+'/training-dataset-with-header.csv',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)


Job Name:  baseline-suggestion-job-2019-12-09-01-09-02-641
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/baselining/data/training-dataset-with-header.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
........................2019-12-09 01:12:48,430 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:143656149352:processing-job/baseline-suggestion-job-2019-12-09-01-09-02-641', 'ProcessingJobName': 'baseline-suggestion-job-2019-12-09-01-09-02-641', 'Environment': {'dataset_format': '{"csv": {"header": 

2019-12-09 01:13:00,048 - bootstrap - INFO - Failed to run /usr/hadoop-3.0.0/bin/yarn --daemon start proxyserver, return code 1
2019-12-09 01:13:00,048 - DefaultDataAnalyzer - INFO - Total number of hosts in the cluster: 1
2019-12-09 01:13:10,054 - DefaultDataAnalyzer - INFO - Running command: bin/spark-submit --master yarn --deploy-mode client --conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider --conf spark.serializer=org.apache.spark.serializer.KryoSerializer /opt/amazon/sagemaker-data-analyzer-1.0-jar-with-dependencies.jar --analytics_input /tmp/spark_job_config.json
2019-12-09 01:13:11 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2019-12-09 01:13:11 INFO  Main:28 - Start analyzing with args: --analytics_input /tmp/spark_job_config.json
2019-12-09 01:13:11 INFO  Main:31 - Analytics input path: DataAnalyzerParams(/tmp/spark_job_config.json)
2019-1

2019-12-09 01:13:42 INFO  YarnClientSchedulerBackend:54 - SchedulerBackend is ready for scheduling beginning after waiting maxRegisteredResourcesWaitingTime: 30000(ms)
2019-12-09 01:13:42 WARN  SparkContext:66 - Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '/tmp' appears to be on the local filesystem.
2019-12-09 01:13:42 INFO  SharedState:54 - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/usr/spark-2.3.1/spark-warehouse').
2019-12-09 01:13:42 INFO  SharedState:54 - Warehouse path is 'file:/usr/spark-2.3.1/spark-warehouse'.
2019-12-09 01:13:42 INFO  StateStoreCoordinatorRef:54 - Registered StateStoreCoordinator endpoint
2019-12-09 01:13:42 INFO  DatasetReader:90 - Files to process:List(file:///opt/ml/processing/input/baseline_dataset_input/training-dataset-with-header.csv)
2019-12-09 01:13:43 INFO  FileSourceStrategy:54 - Pruning directories with: 
2019-12-09 01:13:43

2019-12-09 01:13:52 INFO  TaskSetManager:54 - Finished task 0.0 in stage 2.0 (TID 2) in 3565 ms on algo-1 (executor 1) (1/1)
2019-12-09 01:13:52 INFO  DAGScheduler:54 - ShuffleMapStage 2 (collect at AnalysisRunner.scala:313) finished in 3.597 s
2019-12-09 01:13:52 INFO  DAGScheduler:54 - looking for newly runnable stages
2019-12-09 01:13:52 INFO  DAGScheduler:54 - running: Set()
2019-12-09 01:13:52 INFO  DAGScheduler:54 - waiting: Set(ResultStage 3)
2019-12-09 01:13:52 INFO  DAGScheduler:54 - failed: Set()
2019-12-09 01:13:52 INFO  YarnScheduler:54 - Removed TaskSet 2.0, whose tasks have all completed, from pool 
2019-12-09 01:13:52 INFO  DAGScheduler:54 - Submitting ResultStage 3 (MapPartitionsRDD[27] at collect at AnalysisRunner.scala:313), which has no missing parents
2019-12-09 01:13:52 INFO  MemoryStore:54 - Block broadcast_6 stored as values in memory (estimated size 800.8 KB, free 1455.5 MB)
2019-12-09 01:13:52 INFO  MemoryStore:54 - Block broadcast_6_piece0 stored as bytes in m

2019-12-09 01:14:02 INFO  TaskSetManager:54 - Finished task 0.0 in stage 7.0 (TID 7) in 1296 ms on algo-1 (executor 1) (1/1)
2019-12-09 01:14:02 INFO  YarnScheduler:54 - Removed TaskSet 7.0, whose tasks have all completed, from pool 
2019-12-09 01:14:02 INFO  DAGScheduler:54 - ShuffleMapStage 7 (countByKey at ColumnProfiler.scala:566) finished in 1.328 s
2019-12-09 01:14:02 INFO  DAGScheduler:54 - looking for newly runnable stages
2019-12-09 01:14:02 INFO  DAGScheduler:54 - running: Set()
2019-12-09 01:14:02 INFO  DAGScheduler:54 - waiting: Set(ResultStage 8)
2019-12-09 01:14:02 INFO  DAGScheduler:54 - failed: Set()
2019-12-09 01:14:02 INFO  DAGScheduler:54 - Submitting ResultStage 8 (ShuffledRDD[53] at countByKey at ColumnProfiler.scala:566), which has no missing parents
2019-12-09 01:14:02 INFO  MemoryStore:54 - Block broadcast_11 stored as values in memory (estimated size 3.2 KB, free 1456.2 MB)
2019-12-09 01:14:02 INFO  MemoryStore:54 - Block broadcast_11_piece0 stored as bytes in 

### Explore the generated constraints and statistics

In [14]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
sagemaker/DEMO-ModelMonitor/baselining/results/constraints.json
 sagemaker/DEMO-ModelMonitor/baselining/results/statistics.json


In [15]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

inferred_type            name  numerical_statistics.common.num_missing  \
0      Integral           Churn                                        0   
1      Integral  Account Length                                        0   
2      Integral   VMail Message                                        0   
3    Fractional        Day Mins                                        0   
4      Integral       Day Calls                                        0   
5    Fractional        Eve Mins                                        0   
6      Integral       Eve Calls                                        0   
7    Fractional      Night Mins                                        0   
8      Integral     Night Calls                                        0   
9    Fractional       Intl Mins                                        0   

   numerical_statistics.common.num_present  \
0                                     2333   
1                                     2333   
2                                     2333   
3                                     2333   
4                                     2333   
5                                     2333   
6                                     2333   
7                                     2333   
8                                     2333   
9                                     2333   

       numerical_statistics.distribution.kll.buckets  \
0  [{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...   
1  [{'lower_bound': 1.0, 'upper_bound': 25.2, 'co...   
2  [{'lower_bound': 0.0, 'upper_bound': 5.1, 'cou...   
3  [{'lower_bound': 0.0, 'upper_bound': 35.08, 'c...   
4  [{'lower_bound': 0.0, 'upper_bound': 16.5, 'co...   
5  [{'lower_bound': 31.2, 'upper_bound': 64.26, '...   
6  [{'lower_bound': 12.0, 'upper_bound': 27.8, 'c...   
7  [{'lower_bound': 23.2, 'upper_bound': 60.37999...   
8  [{'lower_bound': 42.0, 'upper_bound': 55.3, 'c...   
9  [{'lower_bound': 0.0, 'upper_bound': 1.8399999...   

   numerical_statistics.distribution.kll.sketch.data  \
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...   
1  [[119.0, 100.0, 111.0, 181.0, 95.0, 104.0, 70....   
2  [[19.0, 0.0, 0.0, 40.0, 36.0, 0.0, 0.0, 24.0, ...   
3  [[178.1, 160.3, 197.1, 105.2, 283.1, 113.6, 23...   
4  [[110.0, 138.0, 117.0, 61.0, 112.0, 87.0, 122....   
5  [[212.8, 221.3, 227.8, 341.3, 286.2, 158.6, 29...   
6  [[100.0, 92.0, 128.0, 79.0, 86.0, 98.0, 112.0,...   
7  [[226.3, 150.4, 214.0, 165.7, 261.7, 187.7, 20...   
8  [[123.0, 120.0, 101.0, 97.0, 129.0, 87.0, 112....   
9  [[10.0, 11.2, 9.3, 6.3, 11.3, 10.5, 0.0, 9.7, ...   

   numerical_statistics.distribution.kll.sketch.parameters.c  \
0                                               0.64           
1                                               0.64           
2                                               0.64           
3                                               0.64           
4                                               0.64           
5                                               0.64           
6                                               0.64           
7                                               0.64           
8                                               0.64           
9                                               0.64           

   numerical_statistics.distribution.kll.sketch.parameters.k  \
0                                             2048.0           
1                                             2048.0           
2                                             2048.0           
3                                             2048.0           
4                                             2048.0           
5                                             2048.0           
6                                             2048.0           
7                                             2048.0           
8                                             2048.0           
9                                             2048.0           

   numerical_s

In [16]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

completeness inferred_type            name  num_constraints.is_non_negative
0           1.0      Integral           Churn                             True
1           1.0      Integral  Account Length                             True
2           1.0      Integral   VMail Message                             True
3           1.0    Fractional        Day Mins                             True
4           1.0      Integral       Day Calls                             True
5           1.0    Fractional        Eve Mins                             True
6           1.0      Integral       Eve Calls                             True
7           1.0    Fractional      Night Mins                             True
8           1.0      Integral     Night Calls                             True
9           1.0    Fractional       Intl Mins                             True

## 2. Analyzing collected data for data quality issues

위의 데이터를 수집한 후, 모니터링 일정(Monitoring Schedules)을 사용하여 데이터를 분석하고 모니터링하세요.

### Create a schedule

In [17]:
# First, copy over some test scripts to the S3 bucket so that they can be used for pre and post processing
boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/preprocessor.py").upload_file('preprocessor.py')
boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/postprocessor.py").upload_file('postprocessor.py')

여러분은 이전에 작성된 엔드포인트에 대한 모델 모니터링 일정을 작성할 수 있습니다. baseline 리소스 (제약 조건 및 통계)를 사용하여 실시간 트래픽과 비교하십시오.

참고로 모니터링 일정을 생성 시 다양한 옵션을 지정할 수 있습니다. 자세한 내용은 https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_monitoring_schedule 을 참조해 주세요.

In [18]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = 'DEMO-xgb-churn-pred-model-monitor-schedule-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    #record_preprocessor_script=pre_processor_script,
    post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,

)


Creating Monitoring Schedule with name: DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58


### Start generating some artificial traffic
아래의 셀은 일부 트래픽을 엔드포인트로 보내는 스레드(thread)를 시작합니다. 이 스레드를 종료하려면 커널을 중지해야 합니다. 트래픽이 없으면 처리할 데이터가 없으므로 모니터링 작업이 실패(`Failed`)로 표시됩니다.

In [24]:
from threading import Thread
from time import sleep
import time

endpoint_name=predictor.endpoint
runtime_client = boto3.client('runtime.sagemaker')

# (just repeating code from above for convenience/ able to run this section independently)
def invoke_endpoint(ep_name, file_name, runtime_client):
    with open(file_name, 'r') as f:
        for row in f:
            payload = row.rstrip('\n')
            response = runtime_client.invoke_endpoint(EndpointName=ep_name,
                                          ContentType='text/csv', 
                                          Body=payload)
            time.sleep(1)
            
def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, 'test_data/test-dataset-input-cols.csv', runtime_client)
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

# Note that you need to stop the kernel to stop the invocations

### Describe and inspect the schedule
describe를 완료했다면, MonitoringScheduleStatus가 Scheduled로 변경되는지 확인하세요.

In [25]:
desc_schedule_result = my_default_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

Schedule status: Scheduled


### List executions
일정(Schedule)은 이전에 지정한 간격으로 작업을 시작합니다. 여기에 최신 5개의 실행(executions)이 나열됩니다. 시간별 일정을 만든 후 이 작업을 시작하면 실행이 비어있을 수 있습니다. 실행이 시작되는 시간 경계(UTC)를 넘을 때까지 기다려야 할 수도 있습니다. 아래 코드에는 대기 로직이 있습니다.

Note: 시간별 일정이라 하더라도 Amazon SageMaker는 20분의 버퍼링 기간으로 실행을 예약합니다. 시간 경계에서 0분에서 20분 사이에 실행이 시작되는 것을 볼 수 있습니다. 이는 백엔드에서 로드 밸런싱을 위해 예상되고 수행됩니다.

In [26]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()    

No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58
We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...
Waiting for the 1st execution to happen...
No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58
Waiting for the 1st execution to happen...
No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58
Waiting for the 1st execution to happen...
No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58
Waiting for the 1st execution to happen...
No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58
Waiting for the 1st execution to happen...
No executions found f

### Inspect a specific execution (latest execution)
여러분은 이전 셀에서 가장 최근에 완료되었거나(latest completed) 실패한 예약 실행(failed scheduled execution)을 선택했습니다. 가능한 터미널 상태(terminal states)와 각각의 의미는 다음과 같습니다.

* `Completed` - 모니터링 실행이 완료되었으며 위반 보고서에 문제가 없음을 의미합니다.
* `CompletedWithViolations` - 실행이 완료되었지만 제약 조건 위반이 감지되었음을 나타냅니다.
* `Failed` - 클라이언트 오류 (잘못된 role 우선 순위) 또는 인프라 문제로 인해 모니터링 실행이 실패했습니다. 정확히 무슨 일이 있었는지 확인하려면 FailureReason 및 ExitMessage에 대한 추가 검사가 필요합니다.
* `Stopped` - 작업이 최대 런타임(max runtime)을 초과했거나 수동으로 중지되었습니다.


In [27]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 60 violations.


In [28]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))

Report Uri: s3://sagemaker-us-east-1-143656149352/sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33/DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58/2019/12/09/02


### List the generated reports

In [29]:
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

Report bucket: sagemaker-us-east-1-143656149352
Report key: sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33/DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58/2019/12/09/02
Found Report Files:
sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33/DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58/2019/12/09/02/constraint_violations.json
 sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33/DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58/2019/12/09/02/constraints.json
 sagemaker/DEMO-ModelMonitor/reports/DEMO-xgb-churn-pred-model-monitor-2019-12-09-00-13-33/DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-09-01-18-58/2019/12/09/02/statistics.json


### Violations report

baseline과 비교하여 위반이 있는 경우 여기에 나열됩니다.

In [30]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', -1)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

constraint_check_type  \
0  data_type_check        
1  data_type_check        
2  data_type_check        
3  data_type_check        
4  data_type_check        
5  data_type_check        
6  data_type_check        
7  data_type_check        
8  data_type_check        
9  data_type_check        

                                                                                                                                            description  \
0  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
1  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
2  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
3  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
4  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
5  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
6  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
7  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
8  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   
9  Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 99.66512107161257% of data is Integral.   

    feature_name  
0  Area Code_510  
1  State_NH       
2  State_NE       
3  State_VA       
4  State_WI       
5  State_ID       
6  State_IA       
7  State_AR       
8  VMail Message  
9  State_OK

### Other commands
모니터링 일정을 시작 및 중지할 수도 있습니다.

In [56]:
#my_default_monitor.stop_monitoring_schedule()
#my_default_monitor.start_monitoring_schedule()

## Delete the resources

데이터 캡처를 계속하기 위해 엔드포인트를 계속 실행할 수 있습니다. 더 많은 데이터를 수집하거나 이 엔드 포인트를 더 사용하지 않으려면 추가 요금이 발생하지 않도록 엔드 포인트를 삭제해야 합니다. 엔드 포인트를 삭제해도 모델 호출 중에 캡처된 데이터는 삭제되지 않습니다. 해당 데이터는 사용자가 직접 삭제할 때까지 Amazon S3에 유지됩니다.

그러나 그 전에 일정을 먼저 삭제해야 한다는 점을 기억해 주세요.

In [57]:
my_default_monitor.delete_monitoring_schedule()
time.sleep(60) # actually wait for the deletion


Deleting Monitoring Schedule with name: DEMO-xgb-churn-pred-model-monitor-schedule-2019-12-08-22-30-35


In [58]:
predictor.delete_endpoint()

In [60]:
predictor.delete_model()

## References
* Amazon SageMaker Model Monitor – Fully Managed Automatic Monitoring For Your Machine Learning Models (https://aws.amazon.com/blogs/aws/amazon-sagemaker-model-monitor-fully-managed-automatic-monitoring-for-your-machine-learning-models/)
* Test data quality at scale with Deequ (https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/)
* Amazon SageMaker Model Monitor Developer Guide (https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-monitor.html)
* KLL sketches paper - Optimal Quantile Approximation in Streams (https://arxiv.org/pdf/1603.05346.pdf)
* Deequ - Unit Tests for Data (https://github.com/awslabs/deequ)